# Working with temperatures

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from geopy import distance
from scipy.optimize import curve_fit
from scipy.signal import filtfilt, butter

## Calculate Cycle of timeseries

### Using Mean

In [ ]:
def calc_cycle_avg(df, target, smooth=True, freq=0.05):
    ''' Calcular o ciclo sasonal de uma variavel
    
    : params :
    df: dataframe com dados de input 
    target (str): nome da coluna alvo
    smooth (bool): dar uma smoothing a media final usando order 3 lowpass 
                   butterworth filter
    freq (float): input para o smoothing
    
    : returns :
    output: dataframe com dados originais mais o ciclo sasonal
    '''
    
    df['yearday'] = df.index.dayofyear
    avg_cycle = df[target].groupby(df['yearday']).mean()
    
    if smooth:  # dar smooth usando filtro lowass butterworth de ordem 3
        smooth_cycle = filtfilt(*butter(3, freq), avg_cycle)
        avg_cycle = pd.Series(smooth_cycle,
                              index=pd.RangeIndex(start=1, stop=367, step=1))
    
    output = df.copy()
    output['seasonal_cycle'] = avg_cycle[df.index.dayofyear].to_numpy()
    
    return output.drop(columns=['yearday'])

### Using polynomial 

In [ ]:
def calc_cycle_poly(df, target, degree, period=365):
    ''' Calcular o ciclo sasonal de uma variavel usando polinomial do tipo 
    y = ... + x^2*b3 + x^1*b4 + b5
    
    : params :
    df: dataframe com dados de input 
    target (str): nome da coluna alvo
    degree (int): ordem do polinomial para usar
    period (int): duracao do perido, default 365 para dados diarios num periodo
                  anual
    
    : returns :
    output: dataframe com dados originais mais o ciclo sasonal)
    '''

    X = [i%period for i in range(0, len(df))]

    coef = np.polyfit(X, df[target].to_numpy(), degree)

    # create curve
    curve = []
    for i in range(len(X)):
        value = coef[-1]
        for d in range(degree):
            value += X[i]**(degree-d)*coef[d]
        curve.append(value)
    
    output = df.copy()
    output['seasonal_cycle'] = curve
    
    return output

### Using a sinusoidal function

In [ ]:
def sine_fit_func(x, b1, b2, b3, b4, period=365):
    ''' funcao sine para fazer fit 
    https://www.hindawi.com/journals/amete/2015/837293/
    '''
    return (b1+(b2*x)+b3*np.sin((2*np.pi/period)*x)+b4*np.cos((2*np.pi/period)*x))


def calc_cycle_sin(df, target, period=365):
    ''' Calcular o ciclo sasonal de uma variavel usando funcao do type sinosoidal
    
    : params :
    df: dataframe com dados de input 
    target (str): nome da coluna alvo
    period (int): duracao do perido, default 365 para dados diarios num periodo
                  anual
    
    : returns :
    output: dataframe com dados originais mais o ciclo sasonal)
    '''
    
    X = np.arange(0, len(df))

    param, _ = curve_fit(sine_fit_func, X, df[target].to_numpy(),
                         p0=[1, 1, 1, 1, period])

    output = df.copy()
    output['seasonal_cycle'] = sine_fit_func(X, *param)
    
    return output